In [27]:
pwd

'/Users/ozkansafak/code/fraktur/notebooks'

In [26]:
%%time
%load_ext autoreload
%autoreload 2

# import standard Python modules
import time 
import numpy as np
import os, sys
import json
import re
import glob
import openai

# # Get the root path of the project
sys.path.append(os.path.abspath(".."))

# Import necessary modules from your src directory
from src.utils import timeit, encode_image, plt
from src.processing import compute_log_spectrum_1d, extract_image_bbox, save_images
from src.api_requests import construct_payload, send_gpt_request
from src.document_generation import save_document
from src.api_requests import single_page

from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% !important; }</style>"))

print('sys.executable:', sys.executable)
print('sys.version:', sys.version, '\n')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<timed exec>:23: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display


sys.executable: /Users/ozkansafak/code/fraktur/.venv/bin/python3
sys.version: 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ] 

CPU times: user 1.69 ms, sys: 1.37 ms, total: 3.06 ms
Wall time: 2.29 ms


---
## Fraktur Translator

In [80]:
# Set the OpenAI API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

# Set the headers for the API request
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}

# Select the value of model_name in ["gpt-4o-2024-08-06", "gpt-4o-mini-2024-07-18"]
model_name = "gpt-4o-2024-08-06"
plotter = False

# Folder containing the Fraktur PDF files
image_path = "../data/Der Weltkrieg v8 Chap 4F (in Fraktur)/*pdf"

# Get the list of PDF file paths
fnames = sorted(glob.glob(image_path))
german_texts = {}
english_texts = {}
total_time = 0 

In [91]:
for i, fname in enumerate(fnames):
    start = time.time()

    pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
    print(f"\n{'>'*10} page:{pageno} (i:{i}/{len(fnames)-1}) ", end="")
    if int(pageno) not in [525, 535]:
        continue
    
    raw_text, raw_german_text, german_text, english_text = single_page(fname, model_name, headers, plotter, pageno)
    
    german_texts[pageno] = german_text
    english_texts[pageno] = english_text
    delta = time.time() - start 
    total_time += delta
    print(f"[time:{delta:.2f}sec] [TotalRuntime:{int(total_time//60)} mins {(total_time - int(total_time//60)*60):.2f} sec] {'>'*10}\n")
    print(raw_text)



>>>>>>>>>> page:456 (i:0/83) 
>>>>>>>>>> page:457 (i:1/83) 
>>>>>>>>>> page:458 (i:2/83) 
>>>>>>>>>> page:459 (i:3/83) 
>>>>>>>>>> page:460 (i:4/83) 
>>>>>>>>>> page:461 (i:5/83) 
>>>>>>>>>> page:462 (i:6/83) 
>>>>>>>>>> page:463 (i:7/83) 
>>>>>>>>>> page:464 (i:8/83) 
>>>>>>>>>> page:465 (i:9/83) 
>>>>>>>>>> page:466 (i:10/83) 
>>>>>>>>>> page:467 (i:11/83) 
>>>>>>>>>> page:468 (i:12/83) 
>>>>>>>>>> page:469 (i:13/83) 
>>>>>>>>>> page:470 (i:14/83) 
>>>>>>>>>> page:471 (i:15/83) 
>>>>>>>>>> page:472 (i:16/83) 
>>>>>>>>>> page:473 (i:17/83) 
>>>>>>>>>> page:474 (i:18/83) 
>>>>>>>>>> page:475 (i:19/83) 
>>>>>>>>>> page:476 (i:20/83) 
>>>>>>>>>> page:477 (i:21/83) 
>>>>>>>>>> page:478 (i:22/83) 
>>>>>>>>>> page:479 (i:23/83) 
>>>>>>>>>> page:480 (i:24/83) 
>>>>>>>>>> page:481 (i:25/83) 
>>>>>>>>>> page:482 (i:26/83) 
>>>>>>>>>> page:483 (i:27/83) 
>>>>>>>>>> page:484 (i:28/83) 
>>>>>>>>>> page:485 (i:29/83) 
>>>>>>>>>> page:486 (i:30/83) 
>>>>>>>>>> page:487 (i:31/83) 
>>>>>>>>>> page:4

In [94]:
# save json outputs
folder_name = 'third_attempt'
with open(f'../output/{folder_name}/english_texts.json', 'w') as f:
    json.dump(english_texts, f)
with open(f'../output/{folder_name}/german_texts.json', 'w') as f:
    json.dump(german_texts, f)

doc1, fname1 = save_document(german_texts, folder_name, language='German')
doc2, fname2 = save_document(english_texts, folder_name, language='English')


``` 
1.  Upload Input folder of pdfs to blob storage.
2.  Read file from s3.
3.  FFT in y -> (x_hi, x_lo), write half_cropped_image to s3
4.  FFT in x -> (y_hi, y_lo), write cropped_image to s3
5.  Read cropped image from s3 -> encode_image -> translate and transcribe -> JSON output

```

### Available models and pricing:
```
"gpt-4o-2024-08-06":
    "price_txt": "$2.50 / 1M input tokens"
    "price_img": "$0.001913 / 1500px^2"
    
"gpt-4o-mini-2024-07-18":
    "price_txt": "$0.150 / 1M input tokens"
    "price_img": "$0.003825 / 1500px^2"
    
```